In [1]:
import numpy as np

from TDOA_estimation import GCC_PHAT, CrossCorr


ModuleNotFoundError: No module named 'TDOA_estimation'

In [23]:
def calculate_tdoa_cross_corr(signal_1, signal_2, fs):
    # Cross-correlation of the two signals
    cross_corr = np.correlate(signal_1, signal_2, mode='full')
    # Time axis for the cross-correlation
    time_axis = np.arange(-len(signal_1) + 1, len(signal_1)) / fs
    # Find the index of the maximum value in the cross-correlation
    
    #plt.plot(np.abs(cross_corr))
    #plt.show()
    
    max_index = np.argmax(np.abs(cross_corr))
    print(max_index)

    # Calculate the TDOA in seconds
    tdoa = time_axis[max_index]
    return tdoa

def gcc_phat_2vecs(sig, refsig, fs=1, max_tau=None, interp=128):
    '''
    This function computes the offset between the signal sig and the reference signal refsig
    using the Generalized Cross Correlation - Phase Transform (GCC-PHAT)method.
    https://github.com/xiongyihui/tdoa/blob/master/gcc_phat.py
    '''
    
    # make sure the length for the FFT is larger or equal than len(sig) + len(refsig)
    n = sig.shape[0] + refsig.shape[0]

    # Generalized Cross Correlation Phase Transform
    SIG = np.fft.rfft(sig, n=n)
    REFSIG = np.fft.rfft(refsig, n=n)
    R = SIG * np.conj(REFSIG)

    cc = np.fft.irfft(R / np.abs(R), n=(interp * n))

    max_shift = int(interp * n / 2)
    if max_tau:
        max_shift = np.minimum(int(interp * fs * max_tau), max_shift)

    cc = np.concatenate((cc[-max_shift:], cc[:max_shift+1]))

    # find max cross correlation index
    #plt.plot(np.abs(cc))
    #plt.show()
    
    shift = np.argmax(np.abs(cc)) - max_shift

    tau = shift / float(interp * fs)
    
    return tau, cc


# Example usage:
if __name__ == "__main__":
    # Generate example signals
    fs = 1000  # Sampling frequency in Hz
    
    length = 1000  # Length of the signal
    frequency = .50  # Frequency of the sinc functions
    offset = 50  # Offset between the sinc functions
    
    x = np.linspace(-length / 2, length / 2, length, endpoint=False)
    sinc_1 = np.abs(np.sinc(frequency * x))
    sinc_2 = ap.abs(np.sinc(frequency * (x - offset*.1)))  # Apply phase shift to the second sinc function
    
    sinc_3 = np.abs(np.sinc(frequency * (x - offset*1.6)))
    sinc_4 = np.abs(np.sinc(frequency * (x - offset*1.7)))  # Apply phase shift to the second sinc function
    
    plt.plot(sinc_1,c= 'b', label = "sinc_1")
    plt.plot(sinc_2, c= 'r', label = "sinc_2")
    plt.plot(sinc_3,c= 'g', label = "sinc_3")
    plt.plot(sinc_4, c= 'y', label = "sinc_4")
    plt.legend()
    plt.show()

    
    # Calculate TDOA using cross-correlation
    startTime = time.time()
    tdoa_cross_corr = calculate_tdoa_cross_corr(sinc_1, sinc_2, fs)
    print("TDOA using cross-correlation:", tdoa_cross_corr, "seconds")
    print("Runtime: ", time.time() - startTime)
    #tdoa_phat_based = phat_tdoa(sinc_1, sinc_2, fs)
    #print("TDOA using phase-based method:", tdoa_phat_based, "seconds")
    
    NewstartTime = time.time()
    offset, _ = gcc_phat_2vecs(sinc_1, sinc_2, fs)
    print("Runtime: ", time.time() - NewstartTime)
    print(offset)

NameError: name 'ap' is not defined

In [21]:
def gcc_phat(channel_matrix, fs, max_tau=None, interp=32):
    '''
    This function computes the offset between the signal sig and the reference signal refsig
    using the Generalized Cross Correlation - Phase Transform (GCC-PHAT)method.
    https://github.com/xiongyihui/tdoa/blob/master/gcc_phat.py
    '''
    
    tau_matrix = np.zeros((4,4))
    #cc_matrix = np.zeros((4,4))
    
    for sig_ind in range(len(channel_matrix)-1):
        for ref_ind in range(sig_ind+1,len(channel_matrix)):
            sig = channel_matrix[sig_ind]
            refsig = channel_matrix[ref_ind]
            
            #return
            # make sure the length for the FFT is larger or equal than len(sig) + len(refsig)
            n = sig.shape[0] + refsig.shape[0]

            # Generalized Cross Correlation Phase Transform
            SIG = np.fft.rfft(sig, n=n)
            REFSIG = np.fft.rfft(refsig, n=n)
            R = SIG * np.conj(REFSIG)

            cc = np.fft.irfft(R / np.abs(R), n=(interp * n))

            max_shift = int(interp * n / 2)
            if max_tau:
                max_shift = np.minimum(int(interp * fs * max_tau), max_shift)

            cc = np.concatenate((cc[-max_shift:], cc[:max_shift+1]))

            # find max cross correlation index
            shift = np.argmax(np.abs(cc)) - max_shift

            tau = shift / float(interp * fs)
            
            tau_matrix[ref_ind,sig_ind] = tau
            #cc_matrix[ref_ind,sig_ind] = cc
    
    return tau_matrix#, cc_matrix

In [22]:
fs = 1000  # Sampling frequency in Hz

length = 50000  # Length of the signal
frequency = .50  # Frequency of the sinc functions
offset = 50  # Offset between the sinc functions

x = np.linspace(-length / 2, length / 2, length, endpoint=False)
sinc_1 = np.sinc(frequency * x)
sinc_2 = np.sinc(frequency * (x - offset*.1))  # Apply phase shift to the second sinc function

sinc_3 = np.sinc(frequency * (x - offset*1.6))
sinc_4 = np.sinc(frequency * (x - offset*1.7))  # Apply phase shift to the second sinc function



tau_mat = gcc_phat(np.vstack(np.array([sinc_1, sinc_2,sinc_3, sinc_4])), fs)
print(tau_mat)

[[ 0.          0.          0.          0.        ]
 [-0.00475     0.          0.          0.        ]
 [-0.08       -0.0749375   0.          0.        ]
 [-0.08496875 -0.08       -0.004625    0.        ]]


In [16]:
def calculate_tdoa_cross_corr(channel_matrix, fs, max_tau=None, interp=16):
    '''
    This function computes the offset between the signal sig and the reference signal refsig
    using the Cross Correlation method.
    '''
    
    tau_matrix = np.zeros((4,4))
    
    
    for sig_ind in range(len(channel_matrix)-1):
        for ref_ind in range(sig_ind+1,len(channel_matrix)):
            sig = channel_matrix[sig_ind]
            refsig = channel_matrix[ref_ind]
    
    
    
            # Cross-correlation of the two signals
            cross_corr = np.correlate(sig, refsig, mode='full')
            # Time axis for the cross-correlation
            time_axis = np.arange(-len(sig) + 1, len(refsig)) / fs
            # Find the index of the maximum value in the cross-correlation

            #plt.plot(np.abs(cross_corr))
            #plt.show()

            max_index = np.argmax(np.abs(cross_corr))
            #print(max_index)

            # Calculate the TDOA in seconds
            tdoa = time_axis[max_index]
            
            tau_matrix[ref_ind,sig_ind] = tdoa
            
    return tau_matrix

In [17]:
tau_matrix = calculate_tdoa_cross_corr(np.vstack(np.array([sinc_1, sinc_2,sinc_3, sinc_4])), fs)
print(tau_matrix)

[[ 0.     0.     0.     0.   ]
 [-0.005  0.     0.     0.   ]
 [-0.08  -0.075  0.     0.   ]
 [-0.085 -0.08  -0.005  0.   ]]
